In [4]:
import tensorflow as tf
import numpy as np
import sklearn.feature_selection as fs

In [48]:
TRAIN_DATA_FILE = "train_data.csv"
TEST_DATA_FILE = "test_data.csv"

training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TRAIN_DATA_FILE,
    target_dtype=np.int,
    features_dtype=np.float32,
    target_column=0
)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TEST_DATA_FILE,
    target_dtype=np.int,
    features_dtype=np.float32,
    target_column=0
)
# print(training_set)
X, y = training_set.data, training_set.target
print(X)
print(y)
X_test, y_test = test_set.data, test_set.target
print(X_test)
print(y_test)


[[  997.9039917    637.5949707    658.66802979 ..., -2203.02001953
  -1180.18994141   433.90600586]
 [  914.19799805   634.23999023   593.70501709 ..., -2250.         -1360.56005859
    524.07501221]
 [ 3800.81005859  1671.33996582  1206.88000488 ...,  2762.57006836
    150.93099976  3800.81005859]
 ..., 
 [ 5833.75976562  3465.73999023  1283.31994629 ...,  3759.70996094
   3267.16992188   388.3460083 ]
 [ 3079.31005859  2435.30004883  1444.40002441 ...,  1880.58996582
   1614.83996582  1518.97998047]
 [ 7352.56982422  6941.18994141  1667.86999512 ...,  5983.12988281
   1249.70996094  2424.22998047]]
[0 0 0 ..., 5 5 5]
[[ 2750.11010742  1361.89001465  -247.50799561 ...,   -81.54180145
   1146.26000977 -1566.48999023]
 [ 2646.25        1760.84997559   481.1000061  ...,   668.75        1519.73999023
   -568.97302246]
 [ 3509.62011719  3509.62011719  -241.32000732 ...,  1132.90002441
    594.059021    -993.65802002]
 ..., 
 [  987.46099854 -2364.60009766  -155.59199524 ..., -4259.29980469

In [5]:
# 重复度检测
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=(.8 * (1 - .8)))  # 去除重复度超过80%的值
sel.fit_transform(X,y)
print(X.shape)
print(y.shape)

(10545, 28)
(10545,)


In [57]:
# 单变量因素
from sklearn.feature_selection import SelectKBest, SelectPercentile

percentile = SelectPercentile(score_func=fs.f_classif, percentile=100)
X_new1 = percentile.fit_transform(X, y) # 保留50%的特征
k_best = SelectKBest(score_func=fs.f_classif, k=20)
X_new2 = k_best.fit_transform(X, y)
print(X_new1.shape)
print(X_new2.shape)

percentile_index = percentile.get_support()
k_best_index = k_best.get_support()


(10545, 28)
(10545, 20)


In [39]:
# 使用模型选择
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
lsvc = LinearSVC(C=0.001, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new3 = model.transform(X)
print(X_new3.shape)

(10545, 28)


In [40]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
print(clf.feature_importances_) 

model = SelectFromModel(clf, prefit=True)
X_new4 = model.transform(X)
print(X_new4.shape)
print(model.get_support()) # 取得被选出的列,就是为True的那些

tree_model_index = model.get_support()

[ 0.14159599  0.05793408  0.04608993  0.03809199  0.03637399  0.01931587
  0.04076579  0.02698541  0.0301383   0.02475596  0.05564788  0.01581794
  0.02571292  0.01788414  0.0356699   0.02358209  0.03453728  0.02494789
  0.0378345   0.02198905  0.03628434  0.03346055  0.02329732  0.01854243
  0.02141414  0.05936967  0.0204838   0.03147685]
(10545, 10)
[ True  True  True  True  True False  True False False False  True False
 False False False False False False  True False  True False False False
 False  True False False]


## 计算哪些特征被选中

In [41]:
# 无奈之举
def get_select_features_index2(X, X_new):
    '''X:传入第一行就行了'''
    index = []
    item = set()
    print(X)
    print(X_new)
    for x in X_new:
        for i, y in enumerate(X):
            if x == y and y not in item:
                item.add(y)
                index.append(i)
    return index
# test
index = get_select_features_index2(X[0], X_new4[0])
print(index)
# 可看到有相同的列

def get_select_features_index(bool_arr):
    '''传入model.get_support()'''
    index = [i for i, b in enumerate(bool_arr) if b == True]
    return index


[  997.9039917    637.5949707    658.66802979 -1882.0300293  -1924.35998535
   997.9039917  -1739.98999023   630.0869751  -1628.23999023 -1325.64001465
  -944.08398438   277.10699463  -206.79899597   536.440979     749.34802246
  -482.99301147   492.00100708   655.77001953  -921.19299316 -1043.16003418
 -1942.48999023   267.13800049   366.60800171   452.23800659   211.32800293
 -2203.02001953 -1180.18994141   433.90600586]
[  997.9039917    637.5949707    658.66802979 -1882.0300293  -1924.35998535
 -1739.98999023  -944.08398438  -921.19299316 -1942.48999023 -2203.02001953]
[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]


## 训练

In [43]:
# 提取其他选出的列

def get_X_new(X, index):
    '''传入被选出的列的下标列表,返回新的数据集'''
    mask = [False] * X.shape[1]
    for i in index:
        mask[i] = True
    return X[:, mask]


In [34]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics



In [62]:
# 通用测试函数

def general_test(feature_index, X_new, classifier):
    features_index = get_select_features_index(feature_index)
    print(features_index)
    
    X_test_r = get_X_new(X_test, features_index)
    print(X_test_r.shape)

    cls = classifier()
    y_pred = cls.fit(X_new, y).predict(X_test_r)
    acc = metrics.accuracy_score(y_test, y_pred)
    print(acc)


In [63]:


general_test(tree_model_index,X_new4,GaussianNB)
general_test(percentile_index,X_new1,GaussianNB)
general_test(k_best_index,X_new2,GaussianNB)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)
0.59
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)
0.686666666667
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)
0.633333333333


[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.59
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.62
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.62


In [65]:
from sklearn.svm import LinearSVC

general_test(tree_model_index,X_new4,LinearSVC)
general_test(percentile_index,X_new1,LinearSVC)
general_test(k_best_index,X_new2,LinearSVC)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.326666666667
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.51
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.416666666667


In [66]:
from sklearn.svm import SVC

general_test(tree_model_index,X_new4,SVC)
general_test(percentile_index,X_new1,SVC)
general_test(k_best_index,X_new2,SVC)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.26
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.26
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.26


In [67]:
from sklearn.ensemble import AdaBoostClassifier

general_test(tree_model_index,X_new4,AdaBoostClassifier)
general_test(percentile_index,X_new1,AdaBoostClassifier)
general_test(k_best_index,X_new2,AdaBoostClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.466666666667
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.526666666667
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.566666666667


In [68]:
from sklearn.ensemble import BaggingClassifier

general_test(tree_model_index,X_new4,BaggingClassifier)
general_test(percentile_index,X_new1,BaggingClassifier)
general_test(k_best_index,X_new2,BaggingClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.583333333333
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.543333333333
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.55


In [69]:
from sklearn.ensemble import ExtraTreesClassifier

general_test(tree_model_index,X_new4,ExtraTreesClassifier)
general_test(percentile_index,X_new1,ExtraTreesClassifier)
general_test(k_best_index,X_new2,ExtraTreesClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)
0.593333333333
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.643333333333
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)
0.623333333333


In [70]:
from sklearn.ensemble import GradientBoostingClassifier

general_test(tree_model_index,X_new4,GradientBoostingClassifier)
general_test(percentile_index,X_new1,GradientBoostingClassifier)
general_test(k_best_index,X_new2,GradientBoostingClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.636666666667
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.593333333333
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.593333333333


In [71]:
from sklearn.ensemble import RandomForestClassifier

general_test(tree_model_index,X_new4,RandomForestClassifier)
general_test(percentile_index,X_new1,RandomForestClassifier)
general_test(k_best_index,X_new2,RandomForestClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.58
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.613333333333
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.606666666667


In [74]:
from sklearn.linear_model import LogisticRegression

general_test(tree_model_index,X_new4,LogisticRegression)
general_test(percentile_index,X_new1,LogisticRegression)
general_test(k_best_index,X_new2,LogisticRegression)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.42
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.483333333333
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.46


In [75]:
from sklearn.linear_model import LogisticRegressionCV

general_test(tree_model_index,X_new4,LogisticRegressionCV)
general_test(percentile_index,X_new1,LogisticRegressionCV)
general_test(k_best_index,X_new2,LogisticRegressionCV)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)


0.513333333333
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)


0.55
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.536666666667


In [77]:
from sklearn.linear_model import PassiveAggressiveClassifier

general_test(tree_model_index,X_new4,PassiveAggressiveClassifier)
general_test(percentile_index,X_new1,PassiveAggressiveClassifier)
general_test(k_best_index,X_new2,PassiveAggressiveClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)
0.17
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)
0.486666666667
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


/usr/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.4


In [79]:
from sklearn.linear_model import RidgeClassifier

general_test(tree_model_index,X_new4,RidgeClassifier)
general_test(percentile_index,X_new1,RidgeClassifier)
general_test(k_best_index,X_new2,RidgeClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)
0.41
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)
0.436666666667
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)
0.443333333333


In [81]:
from sklearn.linear_model import RidgeClassifierCV

general_test(tree_model_index,X_new4,RidgeClassifierCV)
general_test(percentile_index,X_new1,RidgeClassifierCV)
general_test(k_best_index,X_new2,RidgeClassifierCV)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)
0.41
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)
0.436666666667
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)


0.443333333333


In [82]:
from sklearn.linear_model import SGDClassifier

general_test(tree_model_index,X_new4,SGDClassifier)
general_test(percentile_index,X_new1,SGDClassifier)
general_test(k_best_index,X_new2,SGDClassifier)

[0, 1, 2, 3, 4, 6, 10, 18, 20, 25]
(300, 10)
0.346666666667
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
(300, 28)
0.473333333333
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26]
(300, 20)
0.383333333333


/usr/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [129]:
# 通用测试函数--基于神经网络
from sklearn.neural_network import MLPClassifier

def general_test_nn(feature_index, X_new, hidden_layer, activation='relu', alpha=0.0001, solver='adam',max_iter=1000):
    features_index = get_select_features_index(feature_index)
    # print(features_index)

    X_test_r = get_X_new(X_test, features_index)
    # print(X_test_r.shape)

    cls = MLPClassifier(hidden_layer_sizes=hidden_layer, activation=activation, solver=solver, alpha=alpha)
    y_pred = cls.fit(X_new, y).predict(X_test_r)
    print(y_pred)
    acc = metrics.accuracy_score(y_test, y_pred)
    print(acc)
    return acc


In [ ]:
# general_test_nn(percentile_index, X_new1, solver='lbfgs', alpha=1e-5, hidden_layer=(30, 100, 6))
# general_test_nn(percentile_index, X_new1, solver='lbfgs', alpha=1e-5, hidden_layer=(30, 100, 6))
# general_test_nn(percentile_index, X_new1, solver='lbfgs', alpha=1e-5, hidden_layer=(30, 100, 6))

# general_test_nn(k_best_index, X_new2, solver='lbfgs', alpha=1e-5, hidden_layer=(30, 50, 6))
# general_test_nn(k_best_index, X_new2, solver='lbfgs', alpha=1e-5, hidden_layer=(30, 40, 6))
# general_test_nn(k_best_index, X_new2, solver='lbfgs', alpha=1e-5, hidden_layer=(30, 60, 6))

max = 0
for i in range(3):
    cc = general_test_nn(k_best_index, X_new2, solver='lbfgs', alpha=1e-5, hidden_layer=(32,16, 32),
                         max_iter=1000)
    max = cc if max < cc else max
print('max is:', max)

# general_test(percentile_index, X_new1, (5,5))
# general_test(k_best_index, X_new2, MLPClassifier)


[5 5 3 3 5 1 1 1 1 5 1 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 5 1 1 1 5 1 1 1 1 1 1 1 2 1 1 1 1 5 1 1 1 3 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 3 1 1 1 1 1 1 5 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 5 5 5 5 5 5 5 3 5 5 5 5 5 5 5 5 5
 5 5 5 5]
0.263333333333


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 4 4 4 3 3 4 4 3 3 4 3 4 4 4 3 4 3
 3 3 4 4]
0.26


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 2 2 1 5 2 2 1 5 1 1 3 3 3 2 1
 2 2 3 5]
0.26
max is: 0.263333333333
